### 1) Function Application and Mapping

In [1]:
import pandas as pd
import numpy as np

In [3]:
np.abs(frame)

,b,d,e
Utah,0.560297,0.070624,0.908412
Ohio,0.183311,1.364165,1.287377
Texas,1.191709,0.799571,0.626756
Oregon,2.391528,0.426523,0.855059


In [2]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list("bde"), index=["Utah", "Ohio", "Texas", "Oregon"])
frame

,b,d,e
Utah,0.560297,-0.070624,-0.908412
Ohio,-0.183311,1.364165,1.287377
Texas,1.191709,0.799571,0.626756
Oregon,2.391528,-0.426523,-0.855059


Método apply

In [4]:
f = lambda x : x.max() - x.min()
frame.apply(f)

b    2.574839
d    1.790688
e    2.195789
dtype: float64

In [5]:
frame.apply(f, axis = 1)

Utah      1.468709
Ohio      1.547476
Texas     0.564954
Oregon    3.246588
dtype: float64

La función pasada para aplicar no necesita devolver un valor escalar; también puede devolver una Serie
con múltiples valores:

In [6]:
def f(x):
    return pd.Series([x.min(), x.max()], index = ["min","max"])

frame.apply(f)

,b,d,e
min,-0.183311,-0.426523,-0.908412
max,2.391528,1.364165,1.287377


In [8]:
frame

,b,d,e
Utah,0.560297,-0.070624,-0.908412
Ohio,-0.183311,1.364165,1.287377
Texas,1.191709,0.799571,0.626756
Oregon,2.391528,-0.426523,-0.855059


También se pueden usar funciones de Python basadas en elementos. Método applymap.

In [11]:
format = lambda x: "%.3f" % x
frame.applymap(format)

,b,d,e
Utah,0.560,-0.071,-0.908
Ohio,-0.183,1.364,1.287
Texas,1.192,0.800,0.627
Oregon,2.392,-0.427,-0.855


In [14]:
frame["e"].map(format)

Utah      -0.908
Ohio       1.287
Texas      0.627
Oregon    -0.855
Name: e, dtype: object

La función asociada con applymap() se aplica a todos los elementos del DataFrame dado y, por lo tanto, el método applymap() se define solo para DataFrames. De manera similar, la función asociada con el método apply() se puede aplicar a todos los elementos de DataFrame o Series, y por lo tanto, el método apply() se define para los objetos Series y DataFrame. El método map() solo se puede definir para objetos Series en Pandas.

**Ejercicios prácticos para aplicar funciones a DF:**

1. Dado el DataFrame, encuentre la forma más óptima de calcular la raíz de la columna experiencia (Exp) guardando el resultado en una nueva columna identificada con "Sq Exp".

In [15]:
data = {"Exp":[4.4,1.0,2.2,5.5,6.9,10.2,1.1],
       "Educ":[1,1,2,3,4,2,3],
       "W":[100,200,130,120,120,400,200]}

data = pd.DataFrame(data)
data

,Exp,Educ,W
0,4.4,1,100
1,1.0,1,200
2,2.2,2,130
3,5.5,3,120
4,6.9,4,120
5,10.2,2,400
6,1.1,3,200


In [16]:
f = lambda x: np.sqrt(x)

data["sq exp"] = data["Exp"].map(f)
data

,Exp,Educ,W,sq exp
0,4.4,1,100,2.097618
1,1.0,1,200,1.000000
2,2.2,2,130,1.483240
3,5.5,3,120,2.345208
4,6.9,4,120,2.626785
5,10.2,2,400,3.193744
6,1.1,3,200,1.048809


2. Modifique el formato de todos los datos del DF para que se encuentres redondeados sin parte decimal; por ejemplo, si aparece el número 4.4 debe quedar 4, si aparece 5.5 debe quedar 6.

In [17]:
format = lambda x: round(x)
data.applymap(format)

,Exp,Educ,W,sq exp
0,4,1,100,2
1,1,1,200,1
2,2,2,130,1
3,6,3,120,2
4,7,4,120,3
5,10,2,400,3
6,1,3,200,1


3. Empleando la función apply cree una función NO anónima (sin emplear lambda), que asigne en una nueva columna identificada como ("Nivel Academico"):
- Si el valor en la columna Educ corresponde a 1 -> Primaria
- Si el valor en la columna Educ corresponde a 2 -> Secundaria
- Si el valor en la columna Educ corresponde a 3 -> Pregrado
- Si el valor en la columna Educ corresponde a 4 -> Posgrado

Nota: es posible realizar la función de asignación en dos líneas sin emplear condicional if ¿cómo lo lograría?

In [21]:
def nivel_academico(educ):
    return ["Primaria", "Secundaria", "Pregrado", "Posgrado"][educ - 1]
    
data["Nivel Académico"] = data["Educ"].apply(nivel_academico)
data

,Exp,Educ,W,sq exp,Nivel Académico
0,4.4,1,100,2.097618,Primaria
1,1.0,1,200,1.000000,Primaria
2,2.2,2,130,1.483240,Secundaria
3,5.5,3,120,2.345208,Pregrado
4,6.9,4,120,2.626785,Posgrado
5,10.2,2,400,3.193744,Secundaria
6,1.1,3,200,1.048809,Pregrado


### 2.) Datos faltantes

**None**

In [22]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

Cuando trabajamos con objetos de python (dtype = object)  cualquier operación en los datos se realizará en el nivel de Python, con mucha más sobrecarga que las operaciones típicamente rápidas que se ven para las matrices con tipos nativos:

In [23]:
for dtype in ["object", "int"]:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
49.6 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.68 ms ± 24.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



El uso de objetos de Python en una matriz también significa que si realiza agregaciones como sum() o min() en una matriz con un valor None, generalmente obtendrá un error:

In [24]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

**NaN**

In [26]:
vals2 = np.array([1, np.nan, 3, 4])
vals2

array([ 1., nan,  3.,  4.])

Debe tener en cuenta que NaN es un poco como un virus de datos: infecta cualquier otro objeto que toca.

In [28]:
0 * np.nan

nan

Tenga en cuenta que esto significa que los agregados sobre los valores están bien definidos (es decir, no dan como resultado un error) pero no siempre son útiles:

In [30]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

In [32]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

NaN y None tienen su lugar, y Pandas está diseñado para manejarlos casi indistintamente, convirtiendo entre ellos cuando corresponda:

In [33]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Por ejemplo, si establecemos un valor en una matriz de enteros en np.nan, se convertirá automáticamente a tipo float para acomodar el NA:

In [34]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [35]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

#### Detectar valores nulos

In [42]:
data = pd.Series([1, np.nan, "hello", None])
data.isnull().sum()

2

In [38]:
data[data.notnull()]

0        1
2    hello
dtype: object

#### Borrar valores nulos

In [40]:
data.dropna()

0        1
2    hello
dtype: object

In [11]:
df = pd.DataFrame([[1, np.nan, 2], [2, 3, 5], [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


No podemos eliminar valores individuales de un DataFrame; solo podemos borrar filas completas o columnas completas.

In [45]:
df.dropna(axis = 1)

,2
0,2
1,5
2,6


Es posible que le interese colocar filas o columnas con todos los valores NA, o la mayoría de los valores NA.

In [46]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [47]:
df.dropna(axis = 1, how = "all")

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [48]:
df.dropna(axis = 1, thresh = 2)

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [50]:
df.dropna(subset=[1,2])

,0,1,2,3
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


#### Relleno de valores nulos

A veces, en lugar de eliminar los valores de NA, prefiere reemplazarlos con un valor válido.

In [2]:
data = pd.Series([1, np.nan, 2, None, 3], index=list("abcde"))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [6]:
data.fillna(0)

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [7]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [8]:
data.fillna(method="ffill")

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [9]:
data.fillna(method="bfill")

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

Para DataFrames, las opciones son similares:

In [12]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [14]:
df.fillna(method="ffill", axis = 0)

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,2.0,4.0,6


In [16]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,0.424426,0.279179,0.200412
1,-0.075610,-1.036742,-0.283424
2,0.782687,-0.301040,-1.607771
3,1.433020,1.099796,-1.349015
4,-0.502096,0.720408,-0.690215
5,-1.042374,1.114666,0.831182
6,-1.417657,1.203511,0.128524


In [17]:
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan

df

,0,1,2
0,0.424426,NaN,NaN
1,-0.075610,NaN,NaN
2,0.782687,NaN,-1.607771
3,1.433020,NaN,-1.349015
4,-0.502096,0.720408,-0.690215
5,-1.042374,1.114666,0.831182
6,-1.417657,1.203511,0.128524


In [18]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.424426,0.500000,0.000000
1,-0.075610,0.500000,0.000000
2,0.782687,0.500000,-1.607771
3,1.433020,0.500000,-1.349015
4,-0.502096,0.720408,-0.690215
5,-1.042374,1.114666,0.831182
6,-1.417657,1.203511,0.128524


In [20]:
df.fillna(method="bfill", limit=2)

,0,1,2
0,0.424426,NaN,-1.607771
1,-0.075610,NaN,-1.607771
2,0.782687,0.720408,-1.607771
3,1.433020,0.720408,-1.349015
4,-0.502096,0.720408,-0.690215
5,-1.042374,1.114666,0.831182
6,-1.417657,1.203511,0.128524


#### Ejercicios de Valores Nulos

1.) Reemplace los valores de la siguiente serie con el promedio. Tenga en cuenta que, normalmente es esto lo que ocurre. 

In [21]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [25]:
data.fillna(round(np.mean(data),2))

0    1.00
1    3.83
2    3.50
3    3.83
4    7.00
dtype: float64

2.) Seleccione únicamente aquellas filas que tengan 2 o más datos no nulos.

In [26]:
df = pd.DataFrame({"A":[1,2,np.nan],
                  "B":[5,np.nan,np.nan],
                  "C":[1,2,3]})
df

,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2
2,NaN,NaN,3


In [27]:
df.dropna(thresh=2)

,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2


### 3.) Datos duplicados 

In [39]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"], "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [40]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [41]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [43]:
data.drop_duplicates(["k1","k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


#### Ejercicio datos duplicados

In [35]:
data = {"id": [1, 2, 3, 4, 5, 6, 7, 7],
        "first_name": ["Sigrid", "Kennedy", "Theodoric", "Sigrid", "Kennedy", "Beatrix", "Olimpia", "Olimpia"],
        "last_name": ["Mannock", "Donnell", "Rivers", "Mannock", "Donnell", "Parlett", "Guenther", "Guenther"],
        "age": [27, 31, 36, 27, 53, 48, 36, 36],
        "amount": [7.17, 1.90, 1.11, 7.17, 1.41, 6.69, 4.62, 4.62]}
df = pd.DataFrame(data, columns = ["id", "first_name", "last_name", "age", "amount"])
df

,id,first_name,last_name,age,amount
0,1,Sigrid,Mannock,27,7.17
1,2,Kennedy,Donnell,31,1.90
2,3,Theodoric,Rivers,36,1.11
3,4,Sigrid,Mannock,27,7.17
4,5,Kennedy,Donnell,53,1.41
5,6,Beatrix,Parlett,48,6.69
6,7,Olimpia,Guenther,36,4.62
7,7,Olimpia,Guenther,36,4.62


1.) ¿Cuántos registros se encuentran totalmente duplicados?

In [36]:
df.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7     True
dtype: bool

2) Elimine los registros duplicados

In [37]:
df.drop_duplicates()

,id,first_name,last_name,age,amount
0,1,Sigrid,Mannock,27,7.17
1,2,Kennedy,Donnell,31,1.90
2,3,Theodoric,Rivers,36,1.11
3,4,Sigrid,Mannock,27,7.17
4,5,Kennedy,Donnell,53,1.41
5,6,Beatrix,Parlett,48,6.69
6,7,Olimpia,Guenther,36,4.62
